<a href="https://colab.research.google.com/github/eyaJLIDI/churn_prediction_NL/blob/main/correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=073b1280d520f5ca27673c89403dff3b3c11ed1d547ef1c4ecd925d9ff6b4ac5
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *
from pyspark.sql.functions import monotonically_increasing_id 
from pyspark.sql import functions as F
from pyspark.sql import Window
import warnings
warnings.simplefilter(action='ignore')
from textwrap import wrap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth', 800)
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from pyspark.ml.classification import GBTClassifier, RandomForestClassifier ,  DecisionTreeClassifier , LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler

# create a Spark session
spark =  SparkSession \
        .builder \
        .appName("NL+ churn prediction") \
        .getOrCreate()

In [3]:
def correlation(df_data):
  print("start correlation")
  from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
  from pyspark.ml.stat import Correlation
  df_data = df_data.na.fill("unknown")
  df_corr = df_data.select("*")
  categorical_columns = [c for c, t in df_corr.dtypes if t =='string' ]
  print(categorical_columns)
  df_corr = df_corr.drop('DATE_DEBUT','DATE_FIN','date')
  indexer = StringIndexer(inputCols=['PROFIL_IMPAYE', 'SEXE', 'CLT_SOLLICITATION_IND', 'PROFIL_TARIFAIRE17', 'MEDIA', 'CLT_SOLLICITATION_IDFM_IND', 'TYPE_RESIL', 'ACTEUR_RESIL', 'CHURN', 'profil_tarifaire46'],outputCols=['PROFIL_IMPAYE_index', 'SEXE_index', 'CLT_SOLLICITATION_IND_index', 'PROFIL_TARIFAIRE17_index', 'MEDIA_index', 'CLT_SOLLICITATION_IDFM_IND_index', 'TYPE_RESIL_index', 'ACTEUR_RESIL_index', 'CHURN_index', 'profil_tarifaire46_index'])
  indexed = indexer.fit(df_corr).transform(df_corr)
  indexed.printSchema()
  indexed = indexed.drop('PROFIL_IMPAYE', 'SEXE', 'CLT_SOLLICITATION_IND', 'PROFIL_TARIFAIRE17', 'MEDIA', 'CLT_SOLLICITATION_IDFM_IND', 'TYPE_RESIL', 'ACTEUR_RESIL', 'CHURN', 'profil_tarifaire46')
  indexed.show(100)
  # convert to vector column first
  vector_col = "corr_features"
  assembler2 = VectorAssembler(inputCols=indexed.columns, outputCol=vector_col)
  indexed_vector = assembler2.transform(indexed).select(vector_col)
  matrix = Correlation.corr(indexed_vector, vector_col).collect()[0][0]
  corrmatrix = matrix.toArray().tolist()
  print(corrmatrix)
  Columns = ['CLT_REF', 'card_number', 'AGE', 'NOMBRE_CONTRAT_IR_SCO','NOMBRE_CONTRAT_IR_ETU', 'NOMBRE_CONTRAT_NA', 'NOMBRE_CONTRAT_NMS','NOMBRE_GRAT', 'NOMBRE_SOL', 'NOMBRE_AME', 'ADR_CODE_POSTAL','PORTEUR_PAYEUR', 'PROFIL_IMPAYE', 'SEXE', 'CLT_SOLLICITATION_IND', 'PROFIL_TARIFAIRE17', 'CLT_SEL_IND', 'MEDIA',
       'CLT_SOLLICITATION_IDFM_IND', 'TYPE_RESIL', 'ACTEUR_RESIL', 'MONTANT23',
       'ANCIENNETE_NLP', 'CONTRATS_PRECEDENTS', 'CHURN',
       'NOMBRE_DEPT_RESIDENCE_MOIS', 'DT_MEAN', 'ECART_TYPE',
       'VOL_INTERACTION', 'VOL_EVT', 'performance', 'VOL_EVT_SIMPLE',
       'VOL_INTERACTION_SIMPLE', 'DT_MEAN_SIMPLE', 'VOL_EVT_STANDARD',
       'VOL_INTERACTION_STANDARD', 'DT_MEAN_STANDARD', 'VOL_EVT_COMPLIQUE',
       'VOL_INTERACTION_COMPLIQUE', 'DT_MEAN_COMPLIQUE', 'VOL_EVT_BLOQ',
       'VOL_INTERACTION_BLOQ', 'DT_MEAN_BLOQ', 'date', 'profil_tarifaire46',
       'nombre', 'montant48', 'montant_offert']
  import pyspark.pandas as ps
  df = df_data.toPandas()
  plt.figure(figsize=(20, 15))
  sns.heatmap(df.corr(method='pearson'), vmin=-1, vmax=1, annot=True)
  df = df.drop(["performance","VOL_EVT","DT_MEAN","VOL_INTERACTION","VOL_INTERACTION_SIMPLE","DATE_DEBUT","DATE_FIN"],axis=1)
  df = df.drop(['VOL_EVT_SIMPLE',
       'DT_MEAN_SIMPLE', 'VOL_EVT_STANDARD', 'VOL_INTERACTION_STANDARD',
       'DT_MEAN_STANDARD','ECART_TYPE'],axis=1)
  print("end correlation")
  return df